In [1]:
import pm4py
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.algo.evaluation.replay_fitness import algorithm as fitness_evaluator
from pm4py.algo.evaluation.precision import algorithm as precision_evaluator
from pm4py.objects.process_tree.obj import ProcessTree, Operator
from sklearn.cluster import KMeans
import numpy as np

In [28]:
class ApproximateInductiveMiner:
    def __init__(self, filter_steps=10):
        self.filter_values = [x / filter_steps for x in range(1, filter_steps)]

    def filter_event_log(self, log, threshold):
        """
        Dynamically filter activities based on occurrence threshold.
        Incorporates adjustment for information loss as per AIM.
        """
        activity_counts = pm4py.get_event_attribute_values(log, "concept:name")
        max_count = max(activity_counts.values())

        filtered_log = []
        for trace in log:
            valid_trace = all(
                activity_counts.get(ev["concept:name"], 0) >= max_count * threshold
                for ev in trace
            )
            if valid_trace:
                filtered_log.append(trace)

        # Adjust quality for information loss
        info_loss_factor = len(filtered_log) / len(log) if log else 1
        return filtered_log, info_loss_factor

    def calculate_relations(self, log):
        """
        Create a directly-follows matrix for the log.
        """
        activities = list(pm4py.get_event_attribute_values(log, "concept:name"))
        matrix = np.zeros((len(activities), len(activities)))

        for trace in log:
            for i in range(len(trace) - 1):
                a = activities.index(trace[i]["concept:name"])
                b = activities.index(trace[i + 1]["concept:name"])
                matrix[a][b] += 1

        return matrix

    def estimate_quality(self, operator, clusters, relations, info_loss_factor):
        """
        Operator-specific quality estimation incorporating information loss.
        """
        cluster_0_indices = np.where(clusters == 0)[0]
        cluster_1_indices = np.where(clusters == 1)[0]

        inter_cluster = relations[np.ix_(cluster_0_indices, cluster_1_indices)].sum()
        intra_cluster = relations[np.ix_(cluster_0_indices, cluster_0_indices)].sum() + \
                        relations[np.ix_(cluster_1_indices, cluster_1_indices)].sum()

        if operator == "sequence":
            quality = inter_cluster / (inter_cluster + intra_cluster + 1)
        elif operator == "parallel":
            quality = 2 * inter_cluster * intra_cluster / (inter_cluster ** 2 + intra_cluster ** 2 + 1)
        elif operator == "exclusive":
            quality = 1 / (inter_cluster + intra_cluster + 1)
        elif operator == "loop":
            quality = inter_cluster / (intra_cluster + 1)
        else:
            quality = 0

        return quality * info_loss_factor

    def cluster_relations(self, relations, operator):
        """
        Perform clustering using KMeans and operator-specific distance measures.
        """
        if relations.shape[0] < 2:
            return np.zeros(relations.shape[0], dtype=int)

        kmeans = KMeans(n_clusters=2, random_state=42)
        clusters = kmeans.fit_predict(relations)

        return clusters

    def find_best_cut(self, log):
        """
        Evaluate cuts dynamically over filter parameters.
        """
        best_cut = None
        best_quality = -float('inf')

        for filter_value in self.filter_values:
            filtered_log, info_loss_factor = self.filter_event_log(log, filter_value)
            if not filtered_log:
                continue

            relations = self.calculate_relations(filtered_log)
            if relations.size == 0:
                continue

            for operator in ["sequence", "parallel", "exclusive", "loop"]:
                clusters = self.cluster_relations(relations, operator)
                if len(set(clusters)) < 2:
                    continue

                quality = self.estimate_quality(operator, clusters, relations, info_loss_factor)
                if quality > best_quality:
                    best_cut = (operator, clusters)
                    best_quality = quality

        return best_cut

    def split_log(self, log, cut):
        """
        Split log into sub-logs based on clustering results.
        """
        left, right = [], []
        operator, clusters = cut

        activity_names = list(pm4py.get_event_attribute_values(log, "concept:name"))
        activity_to_cluster = {activity_names[i]: clusters[i] for i in range(len(clusters))}

        for trace in log:
            left_trace, right_trace = [], []
            for ev in trace:
                cluster_index = activity_to_cluster.get(ev["concept:name"], -1)
                if cluster_index == 0:
                    left_trace.append(ev)
                elif cluster_index == 1:
                    right_trace.append(ev)

            if left_trace:
                left.append(left_trace)
            if right_trace:
                right.append(right_trace)

        return left, right

    def discover_process_tree(self, log):
        base_case = self.check_base_case(log)
        if base_case:
            return base_case

        cut = self.find_best_cut(log)
        if cut:
            left_log, right_log = self.split_log(log, cut)
            if not left_log or not right_log:
                return self.create_flower_model(log)

            left_tree = self.discover_process_tree(left_log)
            right_tree = self.discover_process_tree(right_log)

            root = ProcessTree(operator=self.get_operator(cut[0]))
            root.children.append(left_tree)
            root.children.append(right_tree)
            return root

        return self.create_flower_model(log)

    def check_base_case(self, log):
        if not log:
            return ProcessTree(operator=Operator.TAU)

        activities = set(ev["concept:name"] for trace in log for ev in trace)
        if len(activities) == 1:
            activity = next(iter(activities))
            return ProcessTree(label=activity)

        return None

    def create_flower_model(self, log):
        """
        Create a flower model as a fallback for unsplittable logs.
        """
        activities = set(ev["concept:name"] for trace in log for ev in trace)
        root = ProcessTree(operator=Operator.PARALLEL)
        for activity in activities:
            root.children.append(ProcessTree(label=activity))
        return root

    def get_operator(self, operator_str):
        """
        Map string operator to ProcessTree operator.
        """
        return {
            "sequence": Operator.SEQUENCE,
            "exclusive": Operator.XOR,
            "parallel": Operator.PARALLEL,
            "loop": Operator.LOOP
        }.get(operator_str, Operator.SEQUENCE)


    def evaluate_model(self, log, process_tree):
        try:
            net, initial_marking, final_marking = pm4py.objects.conversion.process_tree.converter.apply(process_tree)

            fitness = fitness_evaluator.apply(log, net, initial_marking, final_marking)
            precision = precision_evaluator.apply(log, net, initial_marking, final_marking)
            size = len(str(process_tree))

            return {
                "fitness": fitness,
                "precision": precision,
                "size": size
            }
        except Exception as e:
            print("Error during evaluation:", e)
            return {
                "fitness": None,
                "precision": None,
                "size": None
            }

In [29]:
log = xes_importer.apply("BPIC15_1.xes")
aim = ApproximateInductiveMiner()
process_tree = aim.discover_process_tree(log)
print(process_tree)
evaluation = aim.evaluate_model(log, process_tree)
print("Evaluation:", evaluation)

parsing log, completed traces ::   0%|          | 0/1199 [00:00<?, ?it/s]

+( +( *( '01_HOOFD_010', '01_HOOFD_011' ), +( +( *( '02_DRZ_010', '01_HOOFD_110_0' ), +( '01_HOOFD_015', *( '01_HOOFD_020', '01_HOOFD_061' ) ) ), '09_AH_I_010' ) ), '04_BPT_005' )


aligning log, completed variants ::   0%|          | 0/1170 [00:00<?, ?it/s]

computing precision with alignments, completed variants ::   0%|          | 0/38934 [00:00<?, ?it/s]

Evaluation: {'fitness': {'percFitTraces': 0.0, 'averageFitness': 0.23241735241366046, 'percentage_of_fitting_traces': 0.0, 'average_trace_fitness': 0.23241735241366046, 'log_fitness': 0.2056513218901047}, 'precision': 0.42448450825866346, 'size': 179}
